In [4]:
#!/usr/bin/env python3
#################################################
# seismic_risk_monitor.py
#
# Real-time Seismic Risk Analysis Pipeline
# - Fetches live USGS Earthquake Feed (GeoJSON)
# - Fetches Natural Earth Populated Places (GeoJSON)
# - Performs Spatial Join to identify cities within 
#   impact radius of recent quakes (> Mag 4.0)
# - Generates Dashboard: Interactive Map + Risk Chart
#
# Written by [Your Name]
# Created [Date]
#################################################

import os
import sys
import datetime
import requests
import pandas as pd
import geopandas as gpd
import folium
import plotly.express as px
from shapely.geometry import Point

# Optional: For displaying results inline if running in a Jupyter Notebook
try:
    from IPython.display import display, Image, HTML
except ImportError:
    display = None

# ---------------------------------------------------------
# CONFIGURATION (Live Data Sources)
# ---------------------------------------------------------
# USGS Feed: M2.5+ Earthquakes, past 7 days
URL_QUAKES = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson"
# Natural Earth: Populated Places (Simple version)
URL_CITIES = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_populated_places_simple.geojson"

OUTPUT_DIR = "risk_analysis_output"
MIN_MAGNITUDE = 4.0   # Filter for significant events
RISK_RADIUS_KM = 50   # Simple impact radius (can be dynamic based on mag)

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

def fetch_live_data(url, name):
    ##########################################
    # Fetches GeoJSON from a URL and returns a GeoDataFrame.
    ##########################################
    print(f"Fetching {name} data from live feed...")
    try:
        gdf = gpd.read_file(url)
        print(f" -> Successfully loaded {len(gdf)} records.")
        return gdf
    except Exception as e:
        print(f"Error fetching {name}: {e}")
        raise

def perform_risk_analysis(quakes_gdf, cities_gdf, min_mag, radius_km):
    ##########################################
    # 1. Filters quakes by magnitude
    # 2. Buffers quakes to create "Risk Zones"
    # 3. Spatially joins Cities to Risk Zones
    # Returns: impacted_cities_gdf (Cities within risk zones)
    ##########################################
    print(f"\nAnalyzing impact (Mag > {min_mag}, Radius {radius_km}km)...")
    
    # 1. Filter Significant Quakes
    sig_quakes = quakes_gdf[quakes_gdf['mag'] >= min_mag].copy()
    
    if sig_quakes.empty:
        print("No significant earthquakes found in this timeframe.")
        return gpd.GeoDataFrame()

    # 2. Create Risk Buffers
    # Note: We reproject to a Projected CRS (EPSG:3857) for accurate buffering in meters,
    # then project back to WGS84 (EPSG:4326).
    sig_quakes = sig_quakes.to_crs(epsg=3857)
    
    # Simple logic: Buffer size based on fixed radius (could be dynamic: mag * 10000)
    sig_quakes['geometry'] = sig_quakes.buffer(radius_km * 1000) 
    
    risk_zones = sig_quakes.to_crs(epsg=4326)

    # 3. Spatial Join: Find cities INSIDE risk zones
    # We use 'inner' join so we only keep cities that match
    cities_w_risk = gpd.sjoin(cities_gdf, risk_zones, how="inner", predicate="intersects")
    
    # Clean up columns for clarity
    cities_w_risk = cities_w_risk.rename(columns={'mag': 'Quake_Magnitude', 'place': 'Quake_Location'})
    
    count = len(cities_w_risk)
    print(f" -> identified {count} cities potentially impacted.")
    return cities_w_risk, risk_zones

def create_dashboard(impact_cities, risk_zones, out_dir):
    #######################################
    # Generates:
    # 1. Interactive Impact Map (HTML)
    # 2. Risk Distribution Chart (HTML + PNG)
    #######################################
    outputs = {}
    
    # --- 1. FOLIUM MAP ---
    print("Generating Interactive Map...")
    # Center map roughly global (0,0) or on the first event
    start_loc = [20, 0]
    m = folium.Map(location=start_loc, zoom_start=2, tiles="CartoDB positron")

    # Add Risk Zones (Polygons)
    folium.GeoJson(
        risk_zones,
        name="Seismic Risk Zones",
        style_function=lambda x: {'fillColor': '#ff0000', 'color': '#ff0000', 'fillOpacity': 0.3, 'weight': 1},
        tooltip=folium.GeoJsonTooltip(fields=['title', 'mag'], aliases=['Event:', 'Magnitude:'])
    ).add_to(m)

    # Add Impacted Cities (Markers)
    for _, row in impact_cities.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=6,
            color='black',
            fill=True,
            fill_color='orange',
            fill_opacity=1.0,
            popup=f"<b>City:</b> {row['name']}<br><b>Near:</b> {row['title']}<br><b>Pop:</b> {row.get('pop_max', 'N/A')}"
        ).add_to(m)

    # Save Map
    map_html = os.path.join(out_dir, 'seismic_risk_map.html')
    m.save(map_html)
    outputs['map_html'] = map_html

    # --- 2. PLOTLY CHART ---
    if not impact_cities.empty:
        print("Generating Statistics Chart...")
        # Top 10 cities by magnitude of nearby quake
        top_cities = impact_cities.sort_values('Quake_Magnitude', ascending=False).head(10)
        
        fig = px.bar(
            top_cities,
            x='name',
            y='Quake_Magnitude',
            color='Quake_Magnitude',
            title='Top Cities in Seismic Zones (by Event Magnitude)',
            labels={'name': 'City', 'Quake_Magnitude': 'Magnitude'},
            color_continuous_scale='RdYlGn_r' # Red = High Mag
        )
        
        # Save Chart
        chart_html = os.path.join(out_dir, 'risk_chart.html')
        fig.write_html(chart_html)
        
        # Static Image Export (Kaleido)
        chart_png = os.path.join(out_dir, 'risk_chart.png')
        try:
            fig.write_image(chart_png)
            outputs['chart_png'] = chart_png
        except Exception:
            pass

    return outputs

def main():
    try:
        print("--- STARTING SEISMIC RISK MONITOR ---")
        
        # 1. Data Ingestion (ETL)
        quakes = fetch_live_data(URL_QUAKES, "USGS Earthquakes")
        cities = fetch_live_data(URL_CITIES, "Global Cities")
        
        # 2. Analysis
        impact_cities, risk_zones = perform_risk_analysis(quakes, cities, MIN_MAGNITUDE, RISK_RADIUS_KM)
        
        # 3. Visualization
        if not impact_cities.empty:
            outs = create_dashboard(impact_cities, risk_zones, OUTPUT_DIR)
            print(f"\nSUCCESS! Dashboard generated in '{OUTPUT_DIR}':")
            for k, v in outs.items():
                print(f" - {k}: {v}")
                
            # NOTEBOOK DISPLAY LOGIC (Bonus)
            if display:
                display(HTML("<h3>Analysis Complete</h3>"))
                if 'chart_png' in outs:
                    display(Image(filename=outs['chart_png']))
                display(HTML(f"<a href='{outs['map_html']}' target='_blank'>Click to Open Interactive Map</a>"))
        else:
            print("\nNo cities impacted by recent quakes. Analysis complete.")

    except Exception as e:
        print(f"\nCRITICAL ERROR: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()

--- STARTING SEISMIC RISK MONITOR ---
Fetching USGS Earthquakes data from live feed...
 -> Successfully loaded 636 records.
Fetching Global Cities data from live feed...
 -> Successfully loaded 243 records.

Analyzing impact (Mag > 4.0, Radius 50km)...
 -> identified 4 cities potentially impacted.
Generating Interactive Map...
Generating Statistics Chart...

SUCCESS! Dashboard generated in 'risk_analysis_output':
 - map_html: risk_analysis_output\seismic_risk_map.html
